In [1]:
!pip3 install sentencepiece
!pip3 install tf-sentencepiece
!pip3 install matplotlib
#!pip install tensorflow==1.13.1

     |████████████████████████████████| 1.0MB 3.4MB/s 
     |████████████████████████████████| 2.4MB 3.3MB/s 


In [2]:
import os, sys
from google.colab import drive
drive.mount('/content/gdrive')
gitDir = "/content/gdrive/My Drive/nlp/"
os.chdir(gitDir + "data/")
print(os.listdir("."))

#sys.path.insert(0,gitDir + "data")

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive
['Kaggle Amazon Fine Food.ipynb', 'Organic Dataset.ipynb', 'cache', 'Amazon Reviews.ipynb', 'Dataset_Amazon_Multilingual.ipynb', 'Amazon Multilingual.ipynb', '__pycache__', 'amazon_multilingual.py']


In [3]:
import tensorflow as tf
print(tf.__version__)
import tensorflow_hub as hub
import numpy as np
import tf_sentencepiece

import amazon_multilingual
from tensorflow.keras.layers import *
from tensorflow.keras.models import *

from tqdm import tqdm

print("Version: ", tf.__version__)
print("Eager mode: ", tf.executing_eagerly())
print("Hub version: ", hub.__version__)
print("GPU is", "available" if tf.test.is_gpu_available() else "NOT AVAILABLE")

W0528 10:59:37.414140 140513954633600 __init__.py:56] Some hub symbols are not available because TensorFlow version is less than 1.14


1.13.1
Version:  1.13.1
Eager mode:  False
Hub version:  0.4.0
GPU is available


## create graph

In [0]:
batchSize = 800

In [5]:
dataset_train, feed_dict_train, length_train = amazon_multilingual.getData("UK", shuffle=True, batchsize=batchSize)

cache/amazon_multilingual_UK_category.npy and cache/amazon_multilingual_UK_labels.npy  exist. Using saved npy files...
Using default buffer size: 3200


In [6]:
dataset_val, feed_dict_val, length_val = amazon_multilingual.getData("DE", shuffle=False, batchsize=batchSize)

cache/amazon_multilingual_DE_category.npy and cache/amazon_multilingual_DE_labels.npy  exist. Using saved npy files...
dataset is not shuffled and prefetched


In [7]:
iterator = tf.data.Iterator.from_structure(dataset_train.output_types, dataset_train.output_shapes)
train_iterator = iterator.make_initializer(dataset_train)
val_iterator = iterator.make_initializer(dataset_val)

Instructions for updating:
Colocations handled automatically by placer.


W0528 10:59:53.471896 140513954633600 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/data/ops/iterator_ops.py:358: colocate_with (from tensorflow.python.framework.ops) is deprecated and will be removed in a future version.
Instructions for updating:
Colocations handled automatically by placer.


In [0]:
class Model:
    def __init__(self, data_X, data_Y, xling=None):
        self.n_class = 35
        if xling is None:
          print("Downloading xling...")
          self.xling = hub.Module("https://tfhub.dev/google/universal-sentence-encoder-xling-many/1", trainable=False) 
        else:
          self.xling = xling
        self.create_architecture(data_X, data_Y)
        
    def create_architecture(self, data_X, data_Y):
        #y_hot = tf.one_hot(data_Y, depth=self.n_class)
        self.logits = self.forward(data_X)
        
        self.loss = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(labels=data_Y, logits=self.logits))
        self.train_op = tf.train.AdamOptimizer(epsilon=1e-03).minimize(self.loss) #learning_rate=0.0001, 

        self.predictions = tf.argmax(self.logits,1)
        #self.acc, self.acc_op = tf.metrics.accuracy(labels=data_Y, predictions=self.predictions)
        
        #a = tf.cast(self.predictions, tf.float64)
        self.accuracy = tf.reduce_mean( tf.cast(tf.equal(self.predictions, tf.cast(data_Y, tf.int64)), tf.float32) )
        
    def forward(self, X):
        embedded_text = self.xling(X)
        
        '''
        output1 = tf.layers.dense(embedded_text, self.n_class*2)
        bn1 = tf.layers.batch_normalization(output1, training=True)
        relu1 = tf.nn.relu(bn1)
        output = tf.layers.dense(relu1, self.n_class)
        '''
        
        d1 = tf.layers.dense(embedded_text, self.n_class)
        #bn1 = tf.layers.batch_normalization(d1, training=True)
        #relu1 = tf.nn.relu(bn1)
            
        return d1

# train

In [0]:
xling = hub.Module("https://tfhub.dev/google/universal-sentence-encoder-xling/en-de/1", trainable=True)

In [10]:
text_input, label = iterator.get_next()
model = Model(text_input, label, xling)

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


I0528 11:00:14.063382 140513954633600 saver.py:1483] Saver not created because there are no variables in the graph to restore


Instructions for updating:
Use keras.layers.dense instead.


W0528 11:00:14.733106 140513954633600 deprecation.py:323] From <ipython-input-8-c7a2d0e01ed2>:34: dense (from tensorflow.python.layers.core) is deprecated and will be removed in a future version.
Instructions for updating:
Use keras.layers.dense instead.


Instructions for updating:
Use tf.cast instead.


W0528 11:00:15.184277 140513954633600 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/math_ops.py:3066: to_int32 (from tensorflow.python.ops.math_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.cast instead.


Instructions for updating:
Deprecated in favor of operator or tf.math.divide.


W0528 11:00:15.280349 140513954633600 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/math_grad.py:199: div (from tensorflow.python.ops.math_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
/usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/gradients_impl.py:110: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


In [0]:
init_op = tf.group([tf.global_variables_initializer(), tf.tables_initializer()])
sess = tf.Session()
sess.run(init_op)

In [0]:
%%time

epochs = 20

loss_hist, acc_hist, val_loss_hist, val_acc_hist = [], [], [], []
loss_hist_epoch, acc_hist_epoch, val_loss_hist_epoch, val_acc_hist_epoch = [], [], [], []

for epoch in range(epochs):
  print('\nEpoch: {}'.format(epoch + 1))
  train_loss, train_accuracy = 0, 0
  val_loss, val_accuracy = 0, 0  
  counter = 0
  
  sess.run(train_iterator, feed_dict=feed_dict_train)
  
  try:
    with tqdm(total = length_train) as pbar:
      while True:
        _, l, a = sess.run([model.train_op, model.loss, model.accuracy])
        #print(a,l)
        train_loss += l
        train_accuracy += a
        loss_hist.append(l)
        acc_hist.append(a)
        pbar.set_postfix_str((l, a, counter))
        pbar.update(batchSize)
        counter += 1
  except tf.errors.OutOfRangeError:
     pass
     print("\tfinished after", counter, "batches.")
      
  
  loss_hist_epoch.append(train_loss / counter)
  acc_hist_epoch.append(train_accuracy / counter)
  #print('\nEpoch: {}'.format(epoch + 1))
  print('Train loss: {:.4f}, acc: {:.4f}'.format(loss_hist_epoch[-1], acc_hist_epoch[-1]))
      
  
  # Validation
  counter = 0
  sess.run(val_iterator, feed_dict=feed_dict_val) 
  try:
    with tqdm(total = length_val) as pbar:
      while True:
        a, l = sess.run([model.accuracy, model.loss])
        val_loss += l
        val_accuracy += a
        val_loss_hist.append(l)
        val_acc_hist.append(a)
        pbar.set_postfix_str((l, a))
        pbar.update(batchSize)
        counter += 1
  except tf.errors.OutOfRangeError:
     pass
     print("\tfinished after", counter, "batches.")
           
  
  val_loss_hist_epoch.append(val_loss / counter)
  val_acc_hist_epoch.append(val_accuracy / counter)
  print('Val loss: {:.4f}, acc: {:.4f}\n'.format(val_loss_hist_epoch[-1], val_acc_hist_epoch[-1]))



Epoch: 1


1706400it [38:30, 606.44it/s, (0.4555521, 0.8333333, 2132)]                             


	finished after 2133 batches.
Train loss: 0.9473, acc: 0.7260


679200it [10:40, 1132.28it/s, (0.32055205, 0.915683)]                            


	finished after 849 batches.
Val loss: 0.6827, acc: 0.8037


Epoch: 2


...

1706400it [38:42, 593.34it/s, (0.563999, 0.8063063, 2132)]                             


	finished after 2133 batches.
Train loss: 0.5467, acc: 0.8028


679200it [10:37, 1131.24it/s, (0.39971444, 0.86003375)]                            


	finished after 849 batches.
Val loss: 0.6325, acc: 0.8180


Epoch: 5


  0%|          | 4800/1705822 [00:04<26:48, 1057.84it/s, (0.83403456, 0.69625, 5)]

In [0]:
import matplotlib.pyplot as plt
import datetime, os

path = '/content/gdrive/My Drive/nlp/blobs/' + datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S") + "/"
os.mkdir(path)
print("saving to:", path)

title = "UK->DE (trainable XLING)"

plt.plot(loss_hist_epoch, label="train_loss")
plt.plot(val_loss_hist_epoch, label="val_loss")
plt.legend()
plt.title(title)
figpath = path + "epoch_loss.png"
#print(figpath)
plt.savefig(figpath)
plt.show()

plt.plot(acc_hist_epoch, label="train_acc")
plt.plot(val_acc_hist_epoch, label="val_acc")
plt.legend()
plt.title(title)
figpath = path + "epoch_acc.png"
#print(figpath)
plt.savefig(figpath)
plt.show()

plt.plot(loss_hist, label="train_loss")
plt.plot(val_loss_hist, label="val_loss")
plt.legend()
plt.title(title)
figpath = path + "hist_loss.png"
#print(figpath)
plt.savefig(figpath)
plt.show()

plt.plot(acc_hist, label="train_acc")
plt.plot(val_acc_hist, label="val_acc")
plt.legend()
plt.title(title)
figpath = path + "hist_acc.png"
#print(figpath)
plt.savefig(figpath)
plt.show()

np.save(path + "loss_hist_epoch.npy", loss_hist_epoch)
np.save(path + "val_loss_hist_epoch.npy", val_loss_hist_epoch)

np.save(path + "acc_hist_epoch.npy", acc_hist_epoch)
np.save(path + "val_acc_hist_epoch.npy", val_acc_hist_epoch)

np.save(path + "loss_hist.npy", loss_hist)
np.save(path + "val_loss_hist.npy", val_loss_hist)

np.save(path + "acc_hist.npy", acc_hist)
np.save(path + "val_acc_hist.npy", val_acc_hist)

In [0]:
fd = {text_input: ["eins","eins","eins"], label: [0,30,30] }

p, a, acc = sess.run([model.predictions, model.acc_op, model.acc], feed_dict=fd)
print(p,a, acc)

#p, a = sess.run([model.predictions, model.accuracy], feed_dict=fd)
#print(p,a)

p, a = sess.run([model.predictions, model.accuracy], feed_dict=fd)
print(a)

## save model
see https://stackoverflow.com/questions/33759623/tensorflow-how-to-save-restore-a-model

https://cv-tricks.com/tensorflow-tutorial/save-restore-tensorflow-models-quick-complete-tutorial/

In [0]:
!ls -alh /content/testsave

In [0]:
#tf.saved_model.simple_save(sess, "/content/testsave", inputs={"text_input": text_input, "label": label}, outputs={"prediction": model.predictions})

In [0]:
!ls -alh /content/saver

In [0]:
saver = tf.train.Saver()
#saver.save(sess, "/content/saver/model.ckpt")